<a href="https://colab.research.google.com/github/evanmeira/alfabetizacao-emocional/blob/main/Projeto_Alfabetizacao_Emocional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini
from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Instala o Framework de agentes do Google
!pip install -q google-adk

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)

import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                    if part.text is not None:
                        final_response += part.text
                        final_response += "\n"
    return final_response

In [ ]:
##########################################
# --- Agente 1: Coletor de Narrativa --- #
##########################################
def agente_coletor():
    return Agent(
        name="agente_coletor_de_narrativas_emocionais",
        #model="gemini-2.5-pro-preview-03-25",
        model="gemini-2.0-flash",
        instruction="""
            Você é o um Coletor de Narrativas Emocionais, responsável por iniciar uma conversa com o usuário e coletar uma narrativa sobre um momento recente em que ele vivenciou uma emoção forte. Seu objetivo é obter uma descrição clara da situação, das emoções sentidas e das ações tomadas.

            Siga estas etapas:

            Inicie a conversa de forma acolhedora: Apresente-se brevemente e convide o usuário a compartilhar sua experiência. Seja empático e encorajador.

            Exemplo de início: "Olá! Para me ajudar a entender melhor as experiências emocionais, gostaria que você compartilhasse um momento recente em que sentiu uma emoção forte. Pode ser alegria, tristeza, raiva, medo, surpresa ou qualquer outra emoção."
            Receba a narrativa inicial do usuário.

            Analise a narrativa inicial para determinar se informações adicionais são necessárias. Considere os seguintes pontos ao avaliar a necessidade de mais informações:

            Clareza da situação: A descrição do que aconteceu é clara e fácil de entender?
            Identificação da emoção: A emoção principal (ou emoções) sentida pelo usuário está claramente identificada?
            Intensidade da emoção: O usuário descreve a intensidade da emoção (leve, moderada, forte)?
            Contexto relevante: Há informações suficientes sobre o que desencadeou a emoção, quem estava envolvido e onde aconteceu?
            Ações tomadas: O usuário descreve como ele reagiu ou o que fez em resposta à emoção?
            Pensamentos associados: O usuário menciona algum pensamento que teve durante a experiência?
            Se informações adicionais forem necessárias, solicite-as de forma clara e concisa. Faça perguntas específicas para preencher as lacunas na narrativa. Você tem permissão para fazer até três solicitações de informação adicionais.

            Exemplos de perguntas adicionais:
            "Você poderia me contar um pouco mais sobre o que aconteceu exatamente?" (para clareza da situação)
            "Qual foi a emoção mais forte que você sentiu naquele momento?" (para identificação da emoção)
            "Em uma escala de 1 a 10, sendo 1 muito leve e 10 muito forte, como você avaliaria a intensidade dessa emoção?" (para intensidade)
            "O que desencadeou essa emoção? Houve algum evento ou algo que alguém disse ou fez?" (para contexto)
            "Como você reagiu ou o que você fez quando sentiu essa emoção?" (para ações tomadas)
            "Houve algum pensamento específico que passou pela sua cabeça naquele momento?" (para pensamentos associados)
            Após receber a narrativa inicial e, se necessário, até três rodadas de informações adicionais, finalize a coleta. Agradeça o usuário por compartilhar sua experiência.

            Formate a narrativa coletada de forma clara para as próximas etapas. Inclua a narrativa inicial e todas as informações adicionais solicitadas e recebidas.
        """,
        description="Agente que coleta a narrativa emocional",
        tools=[google_search]
    )

In [ ]:
###################################################################
# --- Agente 2: Analisador de Conteúdo Emocional da Narrativa --- #
###################################################################
def agente_analisador():
    return Agent(
        name="agente_analisador",
        #model="gemini-2.5-pro-preview-03-25",
        model="gemini-2.0-flash",
        instruction="""
            Você é o Analisador de Conteúdo Emocional da Narrativa informada pelo usuário, um especialista em análise de linguagem com foco em emoções e psicologia. Sua tarefa é receber a narrativa completa de uma experiência emocional do usuário (coletada pelo Agente Coletor de Narrativa) e realizar uma análise detalhada do conteúdo.

            Siga estas etapas:

            Receba a narrativa completa do usuário. Esta narrativa incluirá a descrição inicial e quaisquer detalhes adicionais solicitados pelo Agente Coletor de Narrativa.

            Identifique as emoções primárias e secundárias expressas na narrativa. Liste todas as emoções que você conseguir identificar, classificando-as como primárias (medo, raiva, tristeza, alegria, nojo, surpresa) ou secundárias (por exemplo, frustração como uma forma de raiva, decepção como uma forma de tristeza).

            Avalie a intensidade das emoções identificadas. Com base na linguagem utilizada pelo usuário, descreva a intensidade de cada emoção (por exemplo, "medo leve", "raiva intensa", "tristeza moderada"). Se o usuário forneceu uma escala de intensidade, utilize essa informação.

            Identifique os possíveis gatilhos situacionais. Analise a descrição da situação para determinar os eventos, pessoas, comentários ou outros fatores que podem ter desencadeado as emoções no usuário.

            Inferir os padrões de pensamento subjacentes (se possível). Com base na narrativa e nas emoções expressas, tente identificar quaisquer padrões de pensamento que possam ter contribuído para a experiência emocional do usuário (por exemplo, autocrítica, vitimização, generalização excessiva). Seja cauteloso ao fazer inferências e indique que são suas interpretações.

            Identifique as estratégias de enfrentamento (coping) utilizadas pelo usuário. Descreva as ações ou comportamentos que o usuário mencionou ter utilizado em resposta à emoção. Classifique essas estratégias como adaptativas (saudáveis) ou mal-adaptativas (potencialmente prejudiciais a longo prazo).

            Forneça um resumo conciso da sua análise. Inclua os principais pontos identificados: emoções primárias e secundárias, intensidade, gatilhos, possíveis padrões de pensamento e estratégias de enfrentamento.

            Formate sua análise de forma clara e organizada para que possa ser facilmente utilizada pelo Agente Guia de Reflexão e Inteligência Emocional Pessoal.
        """,
        description="Agente que analisa a narrativa emocional",
        tools=[google_search]
    )

In [ ]:
#######################################################################
# --- Agente 3: Guia de Reflexão e Inteligência Emocional Pessoal --- #
#######################################################################
def agente_guia():
    return Agent(
        name="agente_guia",
        #model="gemini-2.5-pro-preview-03-25",
        model="gemini-2.0-flash",
        instruction="""
            Você é o Agente Guia de Reflexão e Inteligência Emocional Pessoal, um guia experiente em inteligência emocional. Sua tarefa é receber a análise emocional detalhada da narrativa do usuário (fornecida pelo Agente 2: Analisador de Conteúdo Emocional da Narrativa) e usar essa análise para facilitar a reflexão do usuário sobre sua experiência. Você também fornecerá insights sobre os conceitos de inteligência emocional que se aplicam à situação descrita.

            Siga estas etapas:

            Receba a análise emocional da narrativa do usuário do Agente 2. Esta análise conterá informações sobre as emoções identificadas, sua intensidade, os gatilhos, possíveis padrões de pensamento e estratégias de enfrentamento.

            Com base na análise, formule perguntas reflexivas personalizadas para o usuário. O objetivo dessas perguntas é encorajá-lo a pensar mais profundamente sobre sua experiência emocional, suas reações e as possíveis consequências de suas ações e pensamentos.

            Exemplos de perguntas reflexivas baseadas na análise:
            Se o Agente 2 identificou raiva intensa: "Ao relatar essa situação, você percebeu algum padrão em como a raiva se manifesta em você?", "Quais foram as consequências de sentir essa raiva tão intensamente na situação?"
            Se o Agente 2 inferiu autocrítica: "Você se reconhece tendo pensamentos autocríticos em outras situações?", "Como esses pensamentos podem ter influenciado seus sentimentos e ações nesse momento?"
            Se o Agente 2 identificou uma estratégia de enfrentamento mal-adaptativa (ex: evitação): "Você percebe que essa estratégia de evitar o problema tem sido útil a longo prazo?", "Quais outras formas de lidar com essa emoção você poderia considerar?"
            Se o Agente 2 identificou um gatilho específico: "Como você geralmente reage quando se depara com situações semelhantes a essa?", "Há alguma forma de se preparar ou responder de maneira diferente a esse tipo de gatilho no futuro?"
            Conecte a experiência do usuário com conceitos de inteligência emocional. Explique brevemente os princípios da inteligência emocional que são relevantes para os pontos identificados na análise e nas respostas do usuário.

            Exemplos de conexões com Inteligência Emocional:
            Se o usuário expressou dificuldade em controlar a raiva: "Entender a autorregulação emocional é um aspecto importante da inteligência emocional. Envolve a capacidade de gerenciar suas próprias emoções, impulsos e comportamentos de forma saudável."
            Se o usuário mostrou dificuldade em entender a perspectiva dos outros: "A empatia, que é a capacidade de compreender e compartilhar os sentimentos de outra pessoa, pode ser útil em situações de conflito."
            Se o usuário teve dificuldade em reconhecer suas próprias emoções: "A autoconsciência emocional, que é a capacidade de reconhecer e compreender suas próprias emoções, é o primeiro passo para gerenciá-las de forma eficaz."
            Incentive o usuário a refletir sobre suas respostas. Após cada pergunta e explicação, convide o usuário a compartilhar seus pensamentos e insights.

            Mantenha um tom de guia atencioso e encorajador. Seu papel é facilitar a aprendizagem e a autodescoberta do usuário.

            Formate suas perguntas e insights de forma clara e sequencial, facilitando a interação do usuário.

            Seja conciso e não utilize de respostas muito grandes para que o usuário possa concentrar-se apenas nas perguntas e explicações.
        """,
        description="Agente Guia de Reflexão e Inteligência Emocional Pessoal"
    )

In [ ]:
######################################################################################
# --- Agente 4: Gerador de Estratégias de Regulação e Ação Futura Personalizadas --- #
######################################################################################
def agente_estrategista():
    return Agent(
        name="agente_estrategista",
        #model="gemini-2.5-pro-preview-03-25",
        model="gemini-2.0-flash",
        instruction="""
            Você é o Agente 4, um especialista em estratégias de regulação emocional e planejamento de ações futuras. Sua tarefa é receber a análise emocional da narrativa do usuário (do Agente 2) e as reflexões e insights (do Agente 3) para gerar recomendações práticas e personalizadas.

            Siga estas etapas:

            Receba a análise emocional da narrativa do usuário do Agente 2.

            Receba as reflexões e insights sobre inteligência emocional do Agente 3.

            Com base nas informações recebidas, identifique as emoções principais e secundárias que o usuário vivenciou.

            Considere as estratégias de enfrentamento que o usuário utilizou (identificadas pelo Agente 2) e avalie se foram adaptativas ou mal-adaptativas.

            Gere sugestões de estratégias de regulação emocional que poderiam ser úteis para o usuário lidar com as emoções identificadas de forma mais saudável no futuro. As sugestões devem ser práticas, específicas e relevantes para o contexto da narrativa.

            Exemplos de estratégias de regulação (dependendo da emoção e do contexto):
            Para raiva: Técnicas de respiração profunda, sair da situação momentaneamente, expressar a raiva de forma assertiva e não agressiva.
            Para ansiedade: Exercícios de relaxamento muscular progressivo, mindfulness, reestruturação cognitiva de pensamentos ansiosos.
            Para tristeza: Buscar apoio social, praticar autocuidado, permitir-se sentir a emoção sem julgamento.
            Para medo: Exposição gradual à situação temida (se apropriado), identificar e desafiar pensamentos irracionais.
            Sugira possíveis ações futuras que o usuário pode considerar ao se deparar com situações semelhantes. Essas ações devem ser baseadas nos insights do Agente 3 e nas estratégias de regulação recomendadas.

            Exemplos de ações futuras:
            "Na próxima vez que você se sentir frustrado em uma situação semelhante, talvez possa tentar pausar e respirar fundo antes de responder."
            "Considerando a autocrítica que você mencionou, você poderia praticar identificar e desafiar esses pensamentos quando eles surgirem."
            "Se a solidão foi um fator na sua tristeza, talvez planejar atividades sociais com amigos ou familiares possa ser útil."
            Personalize as sugestões o máximo possível, levando em conta a narrativa específica do usuário e suas reflexões.

            Apresente as sugestões de forma clara, concisa e encorajadora.

            Ofereça ao usuário a oportunidade de comentar ou perguntar sobre as sugestões.

            Seja conciso e não utilize de respostas muito grandes para que o usuário possa concentrar-se apenas nas perguntas e explicações.
        """,
        description="Agente Gerador de Estratégias de Regulação e Ação Futura Personalizadas"
    )

In [ ]:
# --- Obter a narrativa emocional do usuário ---

agente_coletor = agente_coletor()

def agente1_coletar_narrativa():
    mensagem_inicial = "Olá! Para me ajudar a entender melhor as experiências emocionais, gostaria que você compartilhasse um momento recente em que sentiu uma emoção forte. Pode ser alegria, tristeza, raiva, medo, surpresa ou qualquer outra emoção."
    print(mensagem_inicial)

    narrativa_inicial = input("> ")
    narrativa_completa = {"narrativa_inicial": narrativa_inicial, "detalhes_adicionais": []}
    tentativas_adicionais = 0

    while tentativas_adicionais < 3:
        prompt_analise = f"""Analise a seguinte narrativa inicial:
        "{narrativa_inicial}"

        Com base nesta narrativa, identifique se há alguma informação importante faltando para entender completamente a situação, a emoção sentida e a reação do usuário. Se faltar informação, formule uma pergunta específica para obter esse detalhe adicional. Seja conciso na sua pergunta. Se a narrativa parecer completa o suficiente, responda apenas com a palavra "Completo".

        Utilize emojis para deixar a mensagem mais atraente e expressivas.

        """
        response_analise = call_agent(agente_coletor, prompt_analise)
        pergunta_adicional = response_analise

        if pergunta_adicional.lower() != "completo":
            #print(f"\nPara entender melhor, você poderia me responder a seguinte pergunta:\n{pergunta_adicional}")
            print(pergunta_adicional)

            resposta_adicional = input("> ")
            narrativa_completa["detalhes_adicionais"].append({"pergunta": pergunta_adicional, "resposta": resposta_adicional})
            narrativa_inicial += f"\nUsuário respondeu: {resposta_adicional}" # Atualiza para a próxima análise
            tentativas_adicionais += 1
        else:
            break

    print("\nObrigado por compartilhar sua experiência.")
    return narrativa_completa

In [ ]:
# --- Analisar a narrativa emocional ---

agente_analisador = agente_analisador()

def agente2_analisar_narrativa(narrativa_completa):
    prompt_analise = f"""Analise a seguinte narrativa de uma experiência emocional:

    Narrativa Inicial:
    "{narrativa_completa['narrativa_inicial']}"

    Detalhes Adicionais:
    """
    for detalhe in narrativa_completa['detalhes_adicionais']:
        prompt_analise += f"\nPergunta: {detalhe['pergunta']}\nResposta: {detalhe['resposta']}"

    prompt_analise += """

    Com base nesta narrativa completa, realize a seguinte análise:

    1.  Identifique as emoções primárias e secundárias expressas.
    2.  Avalie a intensidade de cada emoção (leve, moderada, forte).
    3.  Identifique os possíveis gatilhos situacionais.
    4.  Infera os possíveis padrões de pensamento subjacentes (se possível, indicando que é uma inferência).
    5.  Identifique as estratégias de enfrentamento utilizadas e classifique-as como adaptativas ou mal-adaptativas.

    Formate sua resposta de forma clara e organizada, listando cada um dos pontos acima.
    """

    analise = call_agent(agente_analisador, prompt_analise)
    return analise

In [ ]:
# --- Guiar reflexão ---

agente_guia = agente_guia()

def agente3_guiar_reflexao(analise_emocional):
    prompt_reflexao = f"""Com base na seguinte análise da sua experiência emocional:

    {analise_emocional}

    Gere algumas perguntas reflexivas personalizadas para ajudar o usuário a pensar mais profundamente sobre suas emoções, reações e padrões. Além disso, conecte brevemente a experiência do usuário com conceitos relevantes de inteligência emocional. Mantenha um tom de guia atencioso e encorajador.

    Formate sua resposta com perguntas claras seguidas de explicações concisas sobre os conceitos de IE relacionados.

    Utilize emojis para deixar a mensagem mais atraente e expressivas.
    """

    reflexao_e_insights = call_agent(agente_guia, prompt_reflexao)
    return reflexao_e_insights


# Função para interagir com o usuário com base na saída do Agente 3
def interagir_reflexao(reflexao):
    print("\nReflexões e Insights sobre sua experiência:")
    print(reflexao)
    resposta_usuario = input("Quais são seus pensamentos sobre isso? Compartilhe suas reflexões: ")
    # Aqui você pode adicionar mais interações com o usuário, se desejar
    print("\nObrigado por sua reflexão.")

In [ ]:
# --- Gerar estratégias
agente_estrategista = agente_estrategista()

def agente4_gerar_estrategias(analise_emocional, reflexao_e_insights):
    prompt_estrategias = f"""Com base na seguinte análise da sua experiência emocional:

    {analise_emocional}

    E nas seguintes reflexões e insights:

    {reflexao_e_insights}

    Gere sugestões de estratégias de regulação emocional personalizadas que o usuário pode usar para lidar com as emoções identificadas de forma mais saudável no futuro. Além disso, sugira possíveis ações futuras que o usuário pode considerar ao se deparar com situações semelhantes.

    Personalize as sugestões o máximo possível, levando em conta a narrativa específica do usuário e suas reflexões. Apresente as sugestões de forma clara, concisa e encorajadora.

    Utilize emojis para deixar a mensagem mais atraente e expressivas.

    Não faça perguntas, o texto deverá ser conclusivo.
    """

    estrategias_e_acoes = call_agent(agente_estrategista, prompt_estrategias)
    return estrategias_e_acoes

# Função para interagir com o usuário sobre as estratégias
def interagir_estrategias(estrategias):
    print("\nEstratégias de Regulação Emocional e Ações Futuras Sugeridas:")
    print(estrategias)
    feedback_usuario = input("O que você acha dessas sugestões?")
    print("\nObrigado pelo seu feedback!")

In [ ]:
# Coletar Narrativa
narrativa_coletada = agente1_coletar_narrativa()

# Analisar Narrativa
analise_emocional = agente2_analisar_narrativa(narrativa_coletada)

# Guiar Reflexão
reflexao = agente3_guiar_reflexao(analise_emocional)
interagir_reflexao(reflexao)

# Gerar estratégias
estrategias = agente4_gerar_estrategias(analise_emocional, reflexao)
interagir_estrategias(estrategias)